In [17]:
import gc

In [1]:
!python --version

Python 3.5.3 :: Anaconda 4.4.0 (64-bit)


In [2]:
RESULTS_DIR = "./results/session-01-KagglePlanetAmazon/"
DATA_DIR = "../../Data/kaggle-planet-amazon/"

In [3]:
!head ../../Data/kaggle-planet-amazon/train_v2.csv

image_name,tags
train_0,haze primary
train_1,agriculture clear primary water
train_2,clear primary
train_3,clear primary
train_4,agriculture clear habitation primary road
train_5,haze primary water
train_6,agriculture clear cultivation primary water
train_7,haze primary
train_8,agriculture clear cultivation primary


In [4]:
import pandas as pd

In [5]:
image_tags = pd.read_csv(DATA_DIR + "train_v2.csv")

In [6]:
image_tags.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [7]:
image_tags["tags"] = image_tags["tags"].map(lambda x: x.split(" "))

In [8]:
image_tags.head()

,image_name,tags
0,train_0,"[haze, primary]"
1,train_1,"[agriculture, clear, primary, water]"
2,train_2,"[clear, primary]"
3,train_3,"[clear, primary]"
4,train_4,"[agriculture, clear, habitation, primary, road]"


In [9]:
from sklearn.preprocessing import MultiLabelBinarizer

In [10]:
mlb = MultiLabelBinarizer()

In [11]:
binarizedLabels = mlb.fit_transform(image_tags["tags"])

In [12]:
mlb.classes_

array(['agriculture', 'artisinal_mine', 'bare_ground', 'blooming',
       'blow_down', 'clear', 'cloudy', 'conventional_mine', 'cultivation',
       'habitation', 'haze', 'partly_cloudy', 'primary', 'road',
       'selective_logging', 'slash_burn', 'water'], dtype=object)

In [13]:
type(binarizedLabels)

numpy.ndarray

In [14]:
binarizedLabels_df = pd.DataFrame(binarizedLabels)

In [15]:
binarizedLabels_df.columns = mlb.classes_

In [16]:
binarizedLabels_df.head()

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
2,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,1,0,0,0,0,1,0,0,0,1,0,0,1,1,0,0,0


In [21]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import cv2
from tqdm import tqdm
# from keras.layers import Convolution2D, MaxPooling2D
# from keras.optimizers import SGD
# from keras.callbacks import ModelCheckpoint
# from sklearn.metrics import matthews_corrcoef
# from sklearn.metrics import hamming_loss
# from keras import backend as K
# K.set_image_dim_ordering('tf')

In [22]:
image_tags.values

array([['train_0', list(['haze', 'primary'])],
       ['train_1', list(['agriculture', 'clear', 'primary', 'water'])],
       ['train_2', list(['clear', 'primary'])],
       ..., 
       ['train_40476', list(['agriculture', 'clear', 'primary'])],
       ['train_40477', list(['agriculture', 'clear', 'primary', 'road'])],
       ['train_40478',
        list(['agriculture', 'cultivation', 'partly_cloudy', 'primary'])]], dtype=object)

In [23]:
x_train = []
y_train = binarizedLabels

for f, tags in tqdm(image_tags.values, miniters=1000):
    img = cv2.imread(DATA_DIR + 'train-jpg/{}.jpg'.format(f))
    x_train.append(cv2.resize(img, (32, 32)))

100%|██████████| 40479/40479 [00:54<00:00, 740.35it/s]


In [25]:
y_train = np.array(y_train, np.uint8)
x_train = np.array(x_train, np.float16) / 255.

In [26]:
# print(x_train.shape)
# print(y_train.shape)

split = 35000
x_train, x_valid, y_train, y_valid = x_train[:split], x_train[split:], y_train[:split], y_train[split:]

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(32, 32, 3)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

model.compile(loss='binary_crossentropy', # We NEED binary here, since categorical_crossentropy l1 norms the output before calculating loss.
              optimizer='adam',
              metrics=['accuracy'])
              
model.fit(x_train, y_train,
          batch_size=128,
          epochs=4,
          verbose=1,
          validation_data=(x_valid, y_valid))
          
from sklearn.metrics import fbeta_score

p_valid = model.predict(x_valid, batch_size=128)

(40479, 32, 32, 3)
(40479, 17)
Train on 35000 samples, validate on 5479 samples
Epoch 1/4
35000/35000 [==============================] - 9s - loss: 0.2476 - acc: 0.9052 - val_loss: 0.1915 - val_acc: 0.9243
Epoch 2/4
35000/35000 [==============================] - 7s - loss: 0.1964 - acc: 0.9234 - val_loss: 0.1751 - val_acc: 0.9309
Epoch 3/4
35000/35000 [==============================] - 7s - loss: 0.1834 - acc: 0.9285 - val_loss: 0.1718 - val_acc: 0.9312
Epoch 4/4
35000/35000 [==============================] - 7s - loss: 0.1766 - acc: 0.9309 - val_loss: 0.1638 - val_acc: 0.9351
[[0 0 0 ..., 0 1 0]
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 1]
 ..., 
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]]
[[  4.99738246e-01   1.77891064e-03   2.36788653e-02 ...,   1.31751429e-02
    1.43982638e-02   2.62324154e-01]
 [  5.76599419e-01   2.78852717e-03   2.42183302e-02 ...,   1.87427159e-02
    1.68734062e-02   2.23156705e-01]
 [  3.06500793e-01   1.59110932e-03   3.63582978e-03 ...,   1.72

In [29]:
p_valid

array([[  4.99738246e-01,   1.77891064e-03,   2.36788653e-02, ...,
          1.31751429e-02,   1.43982638e-02,   2.62324154e-01],
       [  5.76599419e-01,   2.78852717e-03,   2.42183302e-02, ...,
          1.87427159e-02,   1.68734062e-02,   2.23156705e-01],
       [  3.06500793e-01,   1.59110932e-03,   3.63582978e-03, ...,
          1.72302639e-03,   6.32462266e-04,   2.70409584e-01],
       ..., 
       [  6.77943408e-01,   1.71829332e-02,   2.33027041e-01, ...,
          4.66450956e-03,   2.70672841e-03,   2.91779935e-01],
       [  7.01241374e-01,   1.23329293e-02,   6.66452870e-02, ...,
          4.93749091e-03,   3.74299614e-03,   3.68487060e-01],
       [  2.71248877e-01,   1.73534674e-03,   5.18756919e-03, ...,
          5.37023460e-03,   3.92480660e-03,   2.22821280e-01]], dtype=float32)

In [31]:
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import hamming_loss

threshold = np.arange(0.1,0.9,0.1)

acc = []
accuracies = []
best_threshold = np.zeros(p_valid.shape[1])
for i in range(p_valid.shape[1]):
    y_prob = np.array(p_valid[:,i])
    for j in threshold:
        y_pred = [1 if prob>=j else 0 for prob in y_prob]
        acc.append(matthews_corrcoef(y_valid[:,i],y_pred))
    acc   = np.array(acc)
    index = np.where(acc==acc.max()) 
    accuracies.append(acc.max()) 
    best_threshold[i] = threshold[index[0][0]]
    acc = []

/home/ec2-user/src/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:516: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(var_yt * var_yp)


In [32]:
best_threshold

array([ 0.4,  0.1,  0.2,  0.1,  0.1,  0.5,  0.6,  0.1,  0.2,  0.3,  0.2,
        0.4,  0.7,  0.4,  0.1,  0.1,  0.3])

In [39]:
print(fbeta_score(y_valid, np.array(p_valid) > 0.4, beta=2, average='samples'))

0.808415320689


In [33]:
y_pred = np.array([[1 if p_valid[i,j]>=best_threshold[j] else 0 for j in range(y_valid.shape[1])] for i in range(len(y_valid))])

In [34]:
hamming_loss(y_valid,y_pred)

0.0775796356140558

In [35]:
total_correctly_predicted = len([i for i in range(len(y_valid)) if (y_valid[i]==y_pred[i]).sum() == 17])

In [36]:
total_correctly_predicted

2350

In [37]:
x_valid.shape

(5479, 32, 32, 3)

In [43]:
(total_correctly_predicted/ x_valid.shape[0]) * 100

42.891038510677134

In [40]:
gc.collect()

0

In [20]:
!nvidia-smi

Wed Aug  2 22:14:07 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.57                 Driver Version: 367.57                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 0000:00:1E.0     Off |                    0 |
| N/A   34C    P8    30W / 149W |      0MiB / 11439MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  